Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Unzip the dataset

In [2]:
import zipfile

In [3]:
dataset_path = '/content/drive/MyDrive/skin cancer dataset.zip'
extracted_path = '/content/dataset'

In [4]:
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [ ]:
#!ls '/content/dataset'

In [5]:
import os
import numpy as np
import tensorflow as tf
import PIL.Image

Define the CNN model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


Compile the model

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Image Preprocessing

In [11]:
def resize_image(image):
    return image.resize((224, 224))

# Split the training images into smaller batches
batch_size = 320

Load the training images and labels

In [12]:
training_images = []
training_labels = []

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/train/benign'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(0)

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/train/malignant'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(1)


Convert the training labels and training images to a NumPy arrays

In [13]:
training_images = np.array([np.array(image) for image in training_images])
training_labels = np.array(training_labels)

Train the model

In [14]:
model.fit(training_images, training_labels, batch_size=32, epochs=1)

301/301 [==============================] - 723s 2s/step - loss: 55.3502 - accuracy: 0.7264


In [15]:
# Save the trained model
model.save('/content/sample_data/melanoma_cancer_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Load the model

In [16]:
from tensorflow.keras.models import load_model

model = load_model('/content/sample_data/melanoma_cancer_model.h5')

Load the new images for testing

In [41]:
import PIL.Image

# new_images = []
# for image_path in ['/path/to/image1.jpg', '/path/to/image2.jpg']:
#   image = PIL.Image.open(image_path)
  # image = resize_and_normalize(image)
  # new_images.append(image)

 # Convert the new images to a NumPy array
  # new_images = np.array(new_images)
def resize_and_normalize(image):
  image = image.resize((224, 224))
  image = np.array(image)
  image = image / 255.0
  return image

# Load the image
image = PIL.Image.open('/content/dataset/melanoma_cancer_dataset/test/malignant/melanoma_10289.jpg')
image = resize_and_normalize(image)

Make predictions


In [42]:
# predictions = model.predict(new_images)

# # Get the predicted labels
# predicted_labels = np.argmax(predictions, axis=1)

prediction = model.predict(np.array([image]))
predicted_label = np.argmax(prediction, axis=1)

1/1 [==============================] - 0s 64ms/step


Print the result

In [43]:
# for predicted_label in predicted_labels:
#   if predicted_label == 0:
#     print('Benign')
#   else:
#     print('Malignant')

if predicted_label == 0:
  print('Benign')
else:
  print('Malignant')

Malignant
